In [2]:
import numpy as np
import pandas as pd
import quandl
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import shrimpy
%matplotlib inline 

In [3]:
#keys da api
public_key = '0bd785ea948a9225b371e6db7f29cf6c4c3597d91ec663bc6b23026bd10ab8f3'
secret_key = '9810f64bf6498a9996d97ea8f8426b0a24b6d2e62163ae780b7e066d2dbe23f1b3be58e5c81580872544d8428b455cd2ed7464b2a27dbfcff75f194afd3cb6f7'
client = shrimpy.ShrimpyApiClient(public_key, secret_key)

In [4]:
#ir buscar os tickers btc , eth , etc e xrp

btc = client.get_candles(
    'binance',   # exchange
    'BTC',      # base_trading_symbol
    'USDT',     # quote_trading_symbol
    '1d'        # interval
)
dates = []
open_data = []
high_data = []
low_data = []
close_data = []

for candle in btc:
    dates.append(candle['time'])
    open_data.append(candle['open'])
    close_data.append(candle['close'])
    
df_btc = pd.DataFrame(btc) 
df_btc['Date'] = df_btc['time'].apply(pd.to_datetime)
df_btc.set_index('Date',inplace=True)





eth = client.get_candles(
    'binance',   # exchange
    'ETH',      # base_trading_symbol
    'USDT',     # quote_trading_symbol
    '1d'        # interval
)

dates = []
open_data = []
high_data = []
low_data = []
close_data = []

for candle in eth:
    dates.append(candle['time'])
    open_data.append(candle['open'])
    close_data.append(candle['close'])
    
df_eth = pd.DataFrame(eth) 
df_eth['Date'] = df_eth['time'].apply(pd.to_datetime)
df_eth.set_index('Date',inplace=True)




xrp = client.get_candles(
    'binance',   # exchange
    'XRP',      # base_trading_symbol
    'USDT',     # quote_trading_symbol
    '1d'        # interval
)
dates = []
open_data = []
high_data = []
low_data = []
close_data = []

for candle in xrp:
    dates.append(candle['time'])
    open_data.append(candle['open'])
    close_data.append(candle['close'])
    
df_xrp = pd.DataFrame(xrp) 
df_xrp['Date'] = df_xrp['time'].apply(pd.to_datetime)
df_xrp.set_index('Date',inplace=True)


etc = client.get_candles(
    'binance',   # exchange
    'ETC',      # base_trading_symbol
    'USDT',     # quote_trading_symbol
    '1d'        # interval
)

dates = []
open_data = []
high_data = []
low_data = []
close_data = []

for candle in etc:
    dates.append(candle['time'])
    open_data.append(candle['open'])
    close_data.append(candle['close'])
    
df_etc = pd.DataFrame(etc) 
df_etc['Date'] = df_etc['time'].apply(pd.to_datetime)
df_etc.set_index('Date',inplace=True)

In [5]:
#ir só buscar as colunas close
btc = df_btc['close']
etc = df_etc['close']
eth = df_eth['close']
xrp = df_xrp['close']

In [6]:
#juntar todas as colunas close na mesma dataframe, embora sejam perdidos muitos dados
portfolio = pd.concat([btc,etc,eth,xrp],axis=1).dropna()
portfolio.columns = ['btc','etc','eth','xrp']

In [7]:
#retorno log
log_ret=np.log(int(portfolio)/int(portfolio).shift(1))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'DataFrame'

In [ ]:
#simulação de montecarlo embora esta parte seja mais para apresentar um grafico final

num_ports = 5000

all_weights = np.zeros((num_ports,len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(4))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [ ]:
def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2] * -1 

In [ ]:
def check_sum(weights):
    #return 0 if the sum of the weights is 1
    return np.sum(weights) -1 

In [ ]:
cons = ({'type':'eq','fun': check_sum})
bounds = ((0, 1), (0, 1), (0, 1), (0, 1))
init_guess = [0.25,0.25,0.25,0.25]

In [ ]:
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [ ]:
opt_results

In [ ]:
#aqui está a array com os pesos da carteira que devemos ter para maximizar o valor do sharpe ratio
opt_results.x

In [ ]:
#a partir desta parte, é codigo para trabalhar a parte ilustrativa de forma a melhor visulizar os resultados 
get_ret_vol_sr(opt_results.x)

In [ ]:
x = get_ret_vol_sr(opt_results.x)[0]
y = get_ret_vol_sr(opt_results.x)[1]

In [ ]:
frontier_y= np.linspace(0,0.3,100)

In [ ]:
def minimize_vol(weights):
    return get_ret_vol_sr(weights)[1]

In [ ]:
frontier_volatility = []
for possible_return in frontier_y:
    cons = ({'type':'eq','fun':check_sum},
            {'type':'eq','fun':lambda w: get_ret_vol_sr(w)[0]-possible_return})
    result = minimize(minimize_vol,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')

plt.plot(frontier_volatility,frontier_y,'g--',linewidth=3,alpha=0.7)

plt.scatter(y,x,c='red',s=50,edgecolors='black')

print('Compra', opt_results.x[0] * 100,'% de Apple')
print('Compra', opt_results.x[1] * 100,'% de Cisco')
print('Compra', opt_results.x[2] * 100,'% de IBM')
print('Compra', opt_results.x[3] * 100,'% de Amazon')


